In [11]:
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime 

def getHoliday(year):
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo"
    api_key_utf8 = 'yNn1SVQGgJeAnb5M8oyK//N9wu0L7h/Ou+1omKfWOhCaZVWeFBvyEdJ0ATHPIp4FYKrIF7jD3CeZF+shPVCEXQ=='
    api_key_decode = parse.unquote(api_key_utf8)

    url_holiday = url + "getRestDeInfo"
    params = {
        'ServiceKey' : api_key_decode,
        'solYear' : year,
        'numOFRows' : 100
    }

    response = requests.get(url, params=params)
    xml = BeautifulSoup(response.text, 'lxml')
    items = xml.find('items')
    item_list = []
    for item in items:
        item_dict = {
            '이름' : item.find('datename').text.strip(),
            '날짜' : datetime.strptime(item.find('locdate').text.strip(), '%Y%m%d')
        }
        item_list.append(item_dict)

    return pd.DataFrame(item_list)

In [12]:
getHoliday(2021)

C:\Users\user\AppData\Local\Temp\ipykernel_24244\2696970768.py:20: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  xml = BeautifulSoup(response.text, 'lxml')


,이름,날짜
0,2·28 민주운동 기념일,2021-02-28
1,납세자의 날,2021-03-03
2,3·8민주의거 기념일,2021-03-08
3,3·15 의거 기념일,2021-03-15
4,상공의 날,2021-03-17
5,서해수호의 날,2021-03-26
6,예비군의 날,2021-04-02
7,4·3희생자 추념일,2021-04-03
8,식목일,2021-04-05
9,보건의 날,2021-04-07
